 Copyright © Sorbonne University.

 This source code is licensed under the MIT license found in the LICENSE file
 in the root directory of this source tree.

# Outlook

In this notebook, we will implement the REINFORCE algorithm using BBRL.

To understand this code, you need to know more about [the BBRL interaction
model](https://github.com/osigaud/bbrl/blob/master/docs/overview.md) Then you
should run [a didactical
example](https://github.com/osigaud/bbrl/blob/master/docs/notebooks/02-multi_env_noautoreset.student.ipynb)
to see how agents interact in BBRL when autoreset=False.

The REINFORCE algorithm is explained in a series of 3 videos: [video
1](https://www.youtube.com/watch?v=R7ULMBXOQtE), [video
2](https://www.youtube.com/watch?v=dKUWto9B9WY) and [video
3](https://www.youtube.com/watch?v=GcJ9hl3T6x8). You can also read the
corresponding slides:
[slides1](http://pages.isir.upmc.fr/~sigaud/teach/ps/3_pg_derivation1.pdf),
[slides2](http://pages.isir.upmc.fr/~sigaud/teach/ps/4_pg_derivation2.pdf),
[slides3](http://pages.isir.upmc.fr/~sigaud/teach/ps/5_pg_derivation3.pdf).

In [1]:
try:
    from easypip import easyimport
except ModuleNotFoundError:
    from subprocess import run

    assert (
        run(["pip", "install", "easypip"]).returncode == 0
    ), "Could not install easypip"
    from easypip import easyimport

easyimport("swig")
easyimport("bbrl_utils").setup()

from typing import Optional, Tuple

import torch
import torch.nn as nn
from bbrl.agents import Agent, Agents, TemporalAgent
from bbrl_utils.algorithms import EpisodicAlgo
from bbrl_utils.nn import build_mlp, setup_optimizer
from bbrl_utils.notebook import setup_tensorboard
from omegaconf import OmegaConf
import os
from bbrl_utils.nn import copy_parameters

c:\Users\asola\.conda\envs\deepdac\lib\site-packages\bbrl_utils\notebook.py:46: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # noqa: F401


# Learning environment

## Configuration

The learning environment is controlled by a configuration that define a few
important things as described in the example below. This configuration can
hold as many extra information as you need, the example below is the minimal
one.

```python
params = {
    # This defines the a path for logs and saved models
    "base_dir": "${gym_env.env_name}/myalgo_${current_time:}",

    # The Gymnasium environment
    "gym_env": {
        "env_name": "CartPoleContinuous-v1",
    },

    # Algorithm
    "algorithm": {
        # Seed used for the random number generator
        "seed": 1023,

        # Number of parallel training environments
        "n_envs": 8,
                
        # Minimum number of steps between two evaluations
        "eval_interval": 500,
        
        # Number of parallel evaluation environments
        "nb_evals": 10,

        # Number of epochs (loops)
        "max_epochs": 40000,

    },
}

# Creates the configuration object, i.e. cfg.algorithm.nb_evals is 10
cfg = OmegaConf.create(params)
```

## The RL algorithm

In this notebook, the RL algorithm is based on `EpisodicAlgo`, that defines
the algorithm environment when using episodes. To use such environment, we
just need to subclass `EpisodicAlgo` and to define two things, namely the
`train_policy` and the `eval_policy`. Both are BBRL agents that, given the
environment state, select the action to perform.

```py
  class MyAlgo(EpisodicAlgo):
      def __init__(self, cfg):
          super().__init__(cfg)

          # Define the train and evaluation policies
          # (the agents compute the workspace `action` variable)
          self.train_policy = MyPolicyAgent(...)
          self.eval_policy = MyEvalAgent(...)

algo = MyAlgo(cfg)
```

The `EpisodicAlgo` defines useful objects:

- `algo.cfg` is the configuration
- `algo.nb_steps` (integer) is the number of steps since the training began
- `algo.logger` is a logger that can be used to collect statistics during training:
    - `algo.logger.add_log("critic_loss", critic_loss, algo.nb_steps)` registers the `critic_loss` value on tensorboard
- `algo.evaluate()` evaluates the current `eval_policy` if needed, and keeps the
agent if it was the best so far (average cumulated reward);
- `algo.visualize_best()` runs the best agent on one episode, and displays the video



Besides, it also defines an `iter_episodes` is simple:

```py
  # With episodes
  for workspace in rl_algo.iter_episodes():
      # workspace is a workspace containing transitions
      # Episodes shorter than the longer one contain duplicated
      # transitions (with `env/done` set to true)
      ...
```

## Definition of agents

The [REINFORCE](https://link.springer.com/content/pdf/10.1007/BF00992696.pdf)
uses a stochastic policy and a baseline which is the value function. Thus we
need an Actor agent, a Critic agent and an Environment agent. The actor agents
are built on an intermediate `ProbAgent`. Two agents that use the output of
`ProbAgent` are defined below:
- `ArgmaxActorAgent` that selects the action with the highest probability
- `StochasticActorAgent` that selects the action using the probability
  distribution

In [2]:
class ProbAgent(Agent):
    # Computes the distribution $p(a_t|s_t)$

    def __init__(self, state_dim, hidden_layers, n_action, name="prob_agent"):
        super().__init__(name)
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [n_action], activation=nn.ReLU()
        )

    def forward(self, t, **kwargs):
        # Get $s_t$
        observation = self.get(("env/env_obs", t))
        # Compute the distribution over actions
        scores = self.model(observation)
        action_probs = torch.softmax(scores, dim=-1)
        assert not torch.any(torch.isnan(action_probs)), "NaN Here"

        self.set(("action_probs", t), action_probs)
        entropy = torch.distributions.Categorical(action_probs).entropy()
        self.set(("entropy", t), entropy)


class StochasticActorAgent(Agent):
    """Sample an action according to $p(a_t|s_t)$"""

    def forward(self, t: int, **kwargs):
        probs = self.get(("action_probs", t))
        action = torch.distributions.Categorical(probs).sample()
        self.set(("action", t), action)


class ArgmaxActorAgent(Agent):
    """Choose an action $a$ that maximizes $p(a_t|s_t)"""

    def forward(self, t: int, *, stochastic: bool = None, **kwargs):
        probs = self.get(("action_probs", t))
        action = probs.argmax(1)
        self.set(("action", t), action)

### VAgent

The VAgent is a neural network which takes an observation as input and whose
output is the value $V(s)$ of this observation. This is useful to
reduce the bias on the estimation of the gradient.

In [3]:
class VAgent(Agent):
    def __init__(self, state_dim, hidden_layers):
        super().__init__()
        self.is_q_function = False
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [1], activation=nn.ReLU()
        )

    def forward(self, t, **kwargs):
        observation = self.get(("env/env_obs", t))
        # The `squeeze(-1)` removes the last dimension of the tensor.
        # (since this is a scalar, we want to ignore this dimension since
        # the target values will also be scalars)
        critic = self.model(observation).squeeze(-1)
        self.set(("v_value", t), critic)

### RL environment

In the next cell, we define the Reinforce environment. It is based on `EpisodicAlgo`
since learning uses full episodes.

In [4]:
class Reinforce(EpisodicAlgo):
    def __init__(self, cfg):
        super().__init__(cfg)

        obs_size, act_size = self.train_env.get_obs_and_actions_sizes()

        # Train and critic agents
        self.proba_agent = ProbAgent(
            obs_size, cfg.algorithm.architecture.actor_hidden_size, act_size
        )
        self.train_policy = Agents(self.proba_agent, StochasticActorAgent())

        # The critic/value agent (if used)
        self.t_critic_agent = TemporalAgent(
            VAgent(obs_size, cfg.algorithm.architecture.critic_hidden_size)
        )

        # Evaluation policy
        self.eval_policy = Agents(self.proba_agent, ArgmaxActorAgent())

        # Setup the optimizer
        self.optimizer = setup_optimizer(
            cfg.optimizer, self.proba_agent, self.t_critic_agent
        )

The next cell describes the arguments of the two main arguments used in the
training function `run`:
- `compute_advantage` computes the reward at each time step
- `compute_critic_loss` computes the loss of the critic (if we use one), i.e.
  the baseline in reinforce

Both functions will be implemented depending on the reinforce flavor,
so you can leave them empty here.

In [5]:
def compute_advantage(
    cfg, reward: torch.Tensor, v_value: torch.Tensor
) -> torch.Tensor:
    """Computes the reward at each episode step
    
    This function is passed as a parameter, and depend on the Reinforce flavor –
    the function in this notebook cell will thus never been called.

    :param reward: The rewards from the environment (tensor TxB)
    :param v_value: The values $V(s)$ computed by the critic (tensor TxB). It
        can None if there is no baseline.
    :returns: The reward (tensor TxB)
    """
    ...


def compute_critic_loss(
    cfg, reward, must_bootstrap, done, v_value
) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
    """Compute the critic loss

    :param reward: The reward from the environment (TxB)
    :param must_bootstrap: Whether the critic should be bootstrapped (TxB)
    :param done: Whether the episode was finished or not at $t$ (TxB)
    :param v_value: The v value computed by the critic ($TxB$)
    :return: The scalar loss
    """
    # By default, we don't have any critic
    # (so just do nothing)
    pass

You can now write the main learning loop, based on the two above functions
(compute_critic_loss can be None if no baseline is used).

$$
\nabla \mathbb{E}_\tau(R(\tau))
\approx
\frac{1}{m} \sum_{i=1}^m \frac{1}{H_i-1} \sum_{t=1}^{H_i-1} A(s_t^{(i)})
\nabla  \log p(a_k^{(i)} | s_k^{(i)})

$$

with $m$ the number of episodes to estimate the gradient, and $H_i$
the number of steps in the episode, and $A$ is the advantage
function `compute_advantage`.

You also need to use `compute_critic_loss` if a baseline is learned
to reduce the variance of the gradient estimator.

In [6]:
def run(reinforce: Reinforce, compute_advantage, compute_critic_loss=None):
    # We work on full episodes here
    for train_workspace in reinforce.iter_episodes():
        # Get relevant tensors (size are time x n_envs x ....)
        terminated, done, action_probs, reward, action = train_workspace[
            "env/terminated",
            "env/done",
            "action_probs",
            "env/reward",
            "action",
        ]
        must_bootstrap = ~terminated
        not_done = ~done
        
        # Implement the main learning loop

        reinforce.t_critic_agent(train_workspace, t=0, n_steps=action.shape[0])
        v_value = train_workspace['v_value']
        
        # critic update
        if compute_critic_loss is not None:
            critic_loss = compute_critic_loss(reinforce.cfg, reward, must_bootstrap, done, v_value)
            reinforce.optimizer.zero_grad()
            critic_loss.backward()
            reinforce.optimizer.step()
            
        reinforce.t_critic_agent(train_workspace, t=0, n_steps=action.shape[0])
        v_value = train_workspace['v_value']
        advantage = compute_advantage(reinforce.cfg, reward, v_value)
        log_pi = torch.log(action_probs.gather(dim=2, index=action.unsqueeze(-1)).squeeze(-1))
        
        # actor update
        actor_loss = -torch.mean(torch.sum(advantage[:-1]*log_pi[:-1]*not_done[:-1], dim=0)/not_done[:-1].sum(dim=0))
        reinforce.logger.add_log("actor_loss", actor_loss, reinforce.nb_steps)
        reinforce.optimizer.zero_grad()
        actor_loss.backward()
        reinforce.optimizer.step()

        reinforce.evaluate()

## Definition of the parameters

In [ ]:
# We first setup tensorboard (it is better to choose "no" when running on your
# own computer)
setup_tensorboard("./outputs/tblogs")

In [7]:
params = {
    "base_dir": "${gym_env.env_name}/reinforce-${variant}-S${algorithm.seed}_${current_time:}",
    "algorithm": {
        # Number of transitions between two evaluations
        "eval_interval": 1000,
        "seed": 1,
        "n_envs": 8,
        "nb_evals": 10,
        "max_epochs": 700,
        "discount_factor": 0.99,
        "critic_coef": 1.0,
        "actor_coef": 1.0,
        "architecture": {
            "actor_hidden_size": [32],
            "critic_hidden_size": [36],
        },
    },
    "gym_env": {
        "env_name": "CartPole-v1",
    },
    "optimizer": {
        "classname": "torch.optim.Adam",
        "lr": 0.001,
    },
}

### First algorithm: summing all the rewards along an episode

The most basic variant of the Policy Gradient algorithms just sums all the
rewards along an episode.

This is implemented with the `apply_sum` function below.

In [8]:
def apply_sum(cfg, reward, *args):
    reward_sum = reward.sum(axis=0)
    reward = torch.zeros_like(reward)
    for i in range(len(reward)):
        reward[i] = reward_sum
    return reward

In [9]:
# Runs and visualize
reinforce_sum = Reinforce(OmegaConf.create({**params, "variant": "sum"}))
run(reinforce_sum, apply_sum)
reinforce_sum.visualize_best()

  0%|          | 0/700 [00:00<?, ?it/s]

Video of best agent recorded in outputs\CartPole-v1\reinforce-sum-S1_20241112-124317\best_agent.mp4
Moviepy - Building video D:\rld\5\outputs\CartPole-v1\reinforce-sum-S1_20241112-124317\best_agent.mp4.
Moviepy - Writing video D:\rld\5\outputs\CartPole-v1\reinforce-sum-S1_20241112-124317\best_agent.mp4



Moviepy - Done !
Moviepy - video ready D:\rld\5\outputs\CartPole-v1\reinforce-sum-S1_20241112-124317\best_agent.mp4


## Exercises

### First algorithm: summing discounted rewards

As explained in the [second
video](https://www.youtube.com/watch?v=dKUWto9B9WY) and [the corresponding
slides](http://pages.isir.upmc.fr/~sigaud/teach/ps/4_pg_derivation2.pdf),
using a discounted reward after the current step and ignoring the rewards
before the current step results in lower variance.

By taking inspiration from the `apply_sum()` function above, code a function
`apply_discounted_sum()` that computes the sum of discounted rewards from
immediate rewards.

Two hints:
- you should proceed backwards, starting from the final step of the episode
  and storing the previous sum into a register
- you need the discount factor as an input to your function.

In [10]:
def apply_discounted_sum(cfg, reward, *args):
    # Implement the function (ignore v_value)
    discounted_sum = torch.zeros_like(reward)
    
    for i in range(len(reward)-2, 0, -1):
        discounted_sum[i] = reward[i+1] + cfg.algorithm.discount_factor*discounted_sum[i+1]

    discounted_sum[0] = reward[1] + cfg.algorithm.discount_factor*discounted_sum[1]

    return discounted_sum 

In [11]:
reinforce_dsum = Reinforce(OmegaConf.create({**params, "variant": "dsum"}))
run(reinforce_dsum, apply_discounted_sum)

  0%|          | 0/700 [00:00<?, ?it/s]

In [12]:
# Visualization

reinforce_dsum.visualize_best()

Video of best agent recorded in outputs\CartPole-v1\reinforce-dsum-S1_20241112-125505\best_agent.mp4
Moviepy - Building video D:\rld\5\outputs\CartPole-v1\reinforce-dsum-S1_20241112-125505\best_agent.mp4.
Moviepy - Writing video D:\rld\5\outputs\CartPole-v1\reinforce-dsum-S1_20241112-125505\best_agent.mp4



Moviepy - Done !
Moviepy - video ready D:\rld\5\outputs\CartPole-v1\reinforce-dsum-S1_20241112-125505\best_agent.mp4


### Second algorithm: Baseline with Temporal Differences

Here, we aim at computing a baseline using temporal differences. The algorithm
for computing the critic loss is given below.

Note the `critic[1:].detach()` in the computation of the temporal difference
target. The idea is that we compute this target as a function of $V(s_{t+1})$,
but we do not want to apply gradient descent on this $V(s_{t+1})$, we will
only apply gradient descent to the $V(s_t)$ according to this target value.

In practice, `x.detach()` detaches a computation graph from a tensor, so it
avoids computing a gradient over this tensor.

Note also the trick to deal with terminal states. If the state is terminal,
$V(s_{t+1})$ does not make sense. Thus we need to ignore this term. So we
multiply the term by `must_bootstrap`: if `must_bootstrap` is True (converted
into an int, it becomes a 1), we get the term. If `must_bootstrap` is False
(=0), we are at a terminal state, so we ignore the term. This trick is used in
many RL libraries, e.g. SB3.

Code a `apply_discounted_sum_minus_baseline()` function, using the critic
learned simultaneously with the policy.

In [13]:
def apply_discounted_sum_minus_baseline(cfg, reward, v_value):
    # Implement the function
    discounted_sum = apply_discounted_sum(cfg, reward)

    discounted_sum_minus_baseline = discounted_sum - v_value
    
    return discounted_sum_minus_baseline


(2) Code a `compute_critic_loss()` using temporal differences (bootstrapped)

In [14]:
def compute_td_critic_loss(cfg, reward, must_bootstrap, done, critic):
    
    target = reward[1:] + cfg.algorithm.discount_factor*critic[1:].detach()*must_bootstrap[1:]
    not_done = (~done)[:-1]
    td = (target - critic[:-1])**2*not_done
    critic_loss = td.sum()/not_done.sum()
    
    return critic_loss

In [15]:
reinforce_td = Reinforce(OmegaConf.create({**params, "variant": "td"}))
run(reinforce_td, apply_discounted_sum_minus_baseline, compute_td_critic_loss)

  0%|          | 0/700 [00:00<?, ?it/s]

In [16]:
# Visualization

reinforce_td.visualize_best()

Video of best agent recorded in outputs\CartPole-v1\reinforce-td-S1_20241112-131529\best_agent.mp4
Moviepy - Building video D:\rld\5\outputs\CartPole-v1\reinforce-td-S1_20241112-131529\best_agent.mp4.
Moviepy - Writing video D:\rld\5\outputs\CartPole-v1\reinforce-td-S1_20241112-131529\best_agent.mp4



Moviepy - Done !
Moviepy - video ready D:\rld\5\outputs\CartPole-v1\reinforce-td-S1_20241112-131529\best_agent.mp4


### Third algorithm: Monte-Carlo Baseline

 The `compute_critic_loss()` function above uses the Temporal Difference
 approach to critic estimation. In this part, we will compare it to using the
 Monte Carlo estimation approach.

As explained in [this video](https://www.youtube.com/watch?v=GcJ9hl3T6x8) and
[these
slides](http://pages.isir.upmc.fr/~sigaud/teach/ps/5_pg_derivation3.pdf), the
MC estimation approach uses the following equation:

$$\phi_{j+1} = \mathop{\mathrm{argmin}}_{\phi_j} \frac{1}{m\times
   H}\sum_{i=1}^m \sum_{t=1}^H \left( \left(\sum_{k=t}^H \gamma^{k-t}
   r(s_k^{(i)},a_k^{(i)}) \right) - \hat{V}^\pi_{\phi_j}(s_t^{(i)}) \right)^2
       $$

The innermost sum of discounted rewards exactly corresponds to the computation
of the `apply_discounted_sum()` function. The rest just consists in computing
the squared difference (also known as the Means Squared Error, or MSE) over
the $m \times H$ samples ($m$ episodes of lenght $H$) that we have collected.

From the above information, create a `compute_critic_loss()` function.

In [17]:
def compute_critic_loss_mc(cfg, reward, must_bootstrap, done, critic):
    
    discounted_sum = apply_discounted_sum(cfg, reward)
    not_done = (~done)[:-1]
    critic_loss = ((discounted_sum[:-1]-critic[:-1])**2 * not_done).sum() / not_done.sum()

    return critic_loss

In [18]:
reinforce_mc = Reinforce(OmegaConf.create({**params, "variant": "mc"}))
run(reinforce_mc, apply_discounted_sum_minus_baseline, compute_critic_loss_mc)

  0%|          | 0/700 [00:00<?, ?it/s]

In [19]:
# Visualization

reinforce_mc.visualize_best()

Video of best agent recorded in outputs\CartPole-v1\reinforce-mc-S1_20241112-133153\best_agent.mp4
Moviepy - Building video D:\rld\5\outputs\CartPole-v1\reinforce-mc-S1_20241112-133153\best_agent.mp4.
Moviepy - Writing video D:\rld\5\outputs\CartPole-v1\reinforce-mc-S1_20241112-133153\best_agent.mp4



Moviepy - Done !
Moviepy - video ready D:\rld\5\outputs\CartPole-v1\reinforce-mc-S1_20241112-133153\best_agent.mp4


Most probably, this will not work well, as initially the learned critic is a
poor estimate of the true $V(s)$. Instead, load an already trained critic that
you have saved after convergence from a previous run, and see if it works
better.

Loading and saving a network or a BBRL agent can easily be performed using
`agent.save(filename)` and `agent.load(filename)`.

Warning: Be cautious with the use of ProbAgent with just a hidden layer,
ProbAgent with build_mlp, and DiscreteActor. Try to be progressive...